In [ ]:
import sys
sys.path.append("../")

import src.paths as PATHS
import src.constants as CONST

import geopandas as gpd
import folium

In [ ]:
def plot_geodataframe_in_folium(gdf: gpd.GeoDataFrame):
    """Make a quick map of the geodataframe."""
    mapa = folium.Map(location=[CONST.CENTRE_NL_LAT, CONST.CENTRE_NL_LON], zoom_start=CONST.DEFAULT_NL_ZOOM, control_scale=True)
    folium.GeoJson(gdf.to_crs(epsg=CONST.EPSG_WGS84)).add_to(mapa)

    return mapa

# Relevant floodplains

Areas chosen to study the river bank erosion.

In [ ]:
areas = gpd.read_file(PATHS.DATA_DIR / "VO155184_Scope_Pilot_Bankerosion_20241129/VO155184_Scope_Pilot_Bankerosion_20241129.shp")
areas.info()

In [ ]:
display(areas)

In [ ]:
for _, row in areas.iterrows():
    print(f"{row['naam'] + ':':<27} {row['geometry'].area / 1e6:>6.2f}km\N{SUPERSCRIPT TWO}")

In [ ]:
m = plot_geodataframe_in_folium(areas)
m

# Etienne's geopackage

In [ ]:
geopackage = PATHS.DATA_DIR / "Levering_erosie_data.gpkg"

gpkg_dict = {}
for _, layer_data in gpd.list_layers(geopackage).iterrows():
    gpkg_dict[layer_data["name"]] = gpd.read_file(geopackage, layer=layer_data["name"])

In [ ]:
gpkg_dict.keys()

In [ ]:
for gdf_name in gpkg_dict:
    print(gdf_name)
    print(gpkg_dict[gdf_name].crs)
    print(gpkg_dict[gdf_name].info())
    print()

In [ ]:
# create a map showing all the data at one, with turn-offable layers
mapa = folium.Map(location=[CONST.CENTRE_NL_LAT, CONST.CENTRE_NL_LON], zoom_start=CONST.DEFAULT_NL_ZOOM, control_scale=True)

for layer in gpkg_dict:
    fg = folium.FeatureGroup(name=layer, show=False).add_to(mapa)
    folium.GeoJson(gpkg_dict[layer]["geometry"].to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg)

fg = folium.FeatureGroup(name="Selected floodplains", show=False).add_to(mapa)
folium.GeoJson(areas.to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg)

folium.LayerControl().add_to(mapa)

mapa

# Luke's draft oevererosie data

In [ ]:
draft_data = PATHS.DATA_DIR / "draft_oever_degradation_data.gpkg"

gpkg_dict = {}
for _, layer_data in gpd.list_layers(draft_data).iterrows():
    gpkg_dict[layer_data["name"]] = gpd.read_file(draft_data, layer=layer_data["name"])

In [ ]:
gpkg_dict.keys()

In [ ]:
for gdf_name in gpkg_dict:
    print(gdf_name)
    print(gpkg_dict[gdf_name].crs)
    print(gpkg_dict[gdf_name].info())
    print()

In [ ]:
# create a map showing all the data at one, with turn-offable layers
mapa = folium.Map(location=[CONST.CENTRE_NL_LAT, CONST.CENTRE_NL_LON], zoom_start=CONST.DEFAULT_NL_ZOOM, control_scale=True)

for layer in gpkg_dict:
    fg = folium.FeatureGroup(name=layer, show=False).add_to(mapa)
    folium.GeoJson(gpkg_dict[layer]["geometry"].to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg)

fg = folium.FeatureGroup(name="Selected floodplains", show=False).add_to(mapa)
folium.GeoJson(areas.to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg)

folium.LayerControl().add_to(mapa)

mapa

In [ ]:
gpkg_dict["vlakken_erosie"]

In [ ]:
gpkg_dict["vlakken_erosie"]["location_id"].value_counts().sort_index()

In [ ]:
gpkg_dict["vlakken_erosie"].groupby(["location_id", "ahn_version_before", "ahn_version_after"])["area"].count()

In [ ]:
gpkg_dict["punten_oever"]

In [ ]:
gpkg_dict["punten_oever"]["location_id"].value_counts().sort_index()

# Luke's econd draft oevererosie data (21jan2025)

In [ ]:
draft_data = PATHS.DATA_DIR / "all_results_20250121_v2.gpkg"

gpkg_dict = {}
for _, layer_data in gpd.list_layers(draft_data).iterrows():
    gpkg_dict[layer_data["name"]] = gpd.read_file(draft_data, layer=layer_data["name"])

In [ ]:
gpkg_dict.keys()

In [ ]:
for gdf_name in gpkg_dict:
    print(gdf_name)
    print(gpkg_dict[gdf_name].crs)
    print(gpkg_dict[gdf_name].info())
    print()

In [ ]:
# create a map showing all the data at one, with turn-offable layers
mapa = folium.Map(location=[CONST.CENTRE_NL_LAT, CONST.CENTRE_NL_LON], zoom_start=CONST.DEFAULT_NL_ZOOM, control_scale=True)

for layer in gpkg_dict:
    fg = folium.FeatureGroup(name=layer, show=False).add_to(mapa)
    folium.GeoJson(gpkg_dict[layer]["geometry"].to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg)

fg = folium.FeatureGroup(name="Selected floodplains", show=False).add_to(mapa)
folium.GeoJson(areas.to_crs(epsg=CONST.EPSG_WGS84)).add_to(fg)

folium.LayerControl().add_to(mapa)

mapa

In [ ]:
fraction_to_test_data = 0.1  # use 10% of the data in the unit tests
random_state = 42  # for reproducibility

for key,output_file in zip(["vlakken_scope", "punten_oever"], ["prediction_regions_for_tests", "river_bank_points_for_tests"]):
    gpkg_dict[key].sample(frac=fraction_to_test_data, random_state=random_state).to_file(f"../tests/assets/{output_file}.geojson", driver="GeoJSON")